# Gera pdfs

## Carrega Pacotes

In [4]:
import pandas as pd
import re
from titlecase import titlecase

## Define Constantes

In [5]:

strlatex = """
  \\documentclass[twoside]{{article}}
  \\usepackage[affil-it]{{authblk}}
  \\usepackage{{lipsum}} % Package to generate dummy text throughout \
this template
  \\usepackage{{eurosym}}
  \\usepackage[sc]{{mathpazo}} % Use the Palatino font
  \\usepackage[T1]{{fontenc}} % Use 8-bit encoding that has 256 glyphs
  \\usepackage[utf8]{{inputenc}}
  \\linespread{{1.05}} % Line spacing-Palatino needs more space between \
lines
  \\usepackage{{microtype}} % Slightly tweak font spacing for \
aesthetics\[IndentingNewLine]
  \\usepackage[hmarginratio=1:1,top=32mm,columnsep=20pt]{{geometry}} % \
Document margins
  \\usepackage{{multicol}} % Used for the two-column layout of the \
document
  \\usepackage[hang,small,labelfont=bf,up,textfont=it,up]{{caption}} % \
Custom captions under//above floats in tables or figures
  \\usepackage{{booktabs}} % Horizontal rules in tables
  \\usepackage{{float}} % Required for tables and figures in the \
multi-column environment-they need to be placed in specific locations \
with the[H] (e.g. \\begin{{table}}[H])
  \\usepackage{{hyperref}} % For hyperlinks in the PDF
  \\usepackage{{lettrine}} % The lettrine is the first enlarged letter \
at the beginning of the text
  \\usepackage{{paralist}} % Used for the compactitem environment which \
makes bullet points with less space between them
  \\usepackage{{abstract}} % Allows abstract customization
  \\renewcommand{{\\abstractnamefont}}{{\\normalfont\\bfseries}} 
  %\\renewcommand{{\\abstracttextfont}}{{\\normalfont\\small\\itshape}} % \
Set the abstract itself to small italic text\[IndentingNewLine]
  \\usepackage{{titlesec}} % Allows customization of titles
  \\renewcommand\\thesection{{\\Roman{{section}}}} % Roman numerals for \
the sections
  \\renewcommand\\thesubsection{{\\Roman{{subsection}}}} % Roman numerals \
for subsections
  \\titleformat{{\\section}}[block]{{\\large\\scshape\\centering}}{{\\\
thesection.}}{{1em}}{{}} % Change the look of the section titles
  \\titleformat{{\\subsection}}[block]{{\\large}}{{\\thesubsection.}}{{1em}}{{}}\
 % Change the look of the section titles
  \\usepackage{{fancyhdr}} % Headers and footers
  \\pagestyle{{fancy}} % All pages have headers and footers
  \\fancyhead{{}} % Blank out the default header
  \\fancyfoot{{}} % Blank out the default footer
  \\fancyhead[C]{{X-meeting $\\bullet$ November 2017 $\\bullet$ S\\~ao \
Pedro}} % Custom header text
  \\fancyfoot[RO,LE]{{}} % Custom footer text
  %----------------------------------------------------------------------------------------
\
  % TITLE SECTION
  %\
----------------------------------------------------------------------------------------\
 
 
 \\title{{\\vspace{{-15mm}}\\fontsize{{24pt}}{{10pt}}\\selectfont\\textbf{{ {} }}}} % Article title
  
  
  \\author{{ {} }}
  
  \\affil{{ {} }}
  \\vspace{{-5mm}}
  \\date{{}}
  
  %----------------------------------------------------------------------------------------\
 
  
  \\begin{{document}}
  
  
  \\maketitle % Insert title
  
  
  \\thispagestyle{{fancy}} % All pages have headers and footers\

  %----------------------------------------------------------------------------------------\
  
  % ABSTRACT
  
  %----------------------------------------------------------------------------------------\
  
  
  \\begin{{abstract}}
  {}
  
  Funding: {} \\\\ 
  \\end{{abstract}}
  \\end{{document}} """



strlatex2 = """\\procpaper[switch=45,
    title={{{}}}, 
    author={{{}}}, 
    index={{{{{}}}}}]
    {{{}}} \n\n """


prefix=""

## Funções Gerais

In [6]:
def namecase(str):
    """Converte Strings para Capitalização Brasileira"""
    lissubs=[(" De "," de "),(" Do "," do "), (" Da ", " da "), (" Das ", " das "),
             (" Dos "," dos ")]
    str2=titlecase(str)
    for subs in lissubs:
        str2=str2.replace(subs[0],subs[1])
    return str2



In [7]:
def brnames(str):
    """Gera Nomes para a regra ABNT"""
    lis=str.split(" ")
    str2=lis[-1]+ ", "
    for word in lis[0:-1]:
        str2=str2+" "+word
    return str2

In [8]:
def orderedunion(lis):
    """Elimina Repetições e mantém a ordem"""
    lis2=[]
    for elem in lis:
        if not (elem in lis2):
            lis2.append(elem)
    return lis2

In [9]:


def fixbin(string,replacements):
    for k, v in replacements.items():
        string=string.replace(k, v)
    return string

## Funções Específicas


In [10]:
def geraAuthors(str):
    """Gera uma Lista de Autores"""
    m = re.findall('[^(),]+\([^()]+[^()]+\)', str)
    return [namecase(word.split("(")[0].strip()) for word in m]

def geraAfil(str):
    """Gera uma lista de Afiliações"""
    m = re.findall('\([^()]+[^()]+\)', str)
    return [inst[1:-1] for inst in m]

In [11]:
def geraStringAfil(lis):
    lisafil=orderedunion(lis)
    stringafil="" 
    i=1
    for inst in lisafil:
        stringafil=stringafil+str(i)+" "+inst+"\n\n"
        i=i+1
    return stringafil
    
def geraStringAuthors(record):
    lisafil=record["Afil"]
    lisauthors=record["Authors"]
    dictauthors={}
    dictafil={}
    lisafilorder=orderedunion(lisafil)
    for i in range(len(lisauthors)):
        dictauthors[lisauthors[i]]=lisafil[i]
    for i in range(len(lisafilorder)):
        dictafil[lisafilorder[i]]=i
    strsaida=""
    for author in lisauthors:
        num=dictafil[dictauthors[author]]
        strsaida=strsaida+author+"$^{"+str(num+1)+"}$, "
    return strsaida[:-1]
    

## Entrada dos Dados

In [12]:
artsds=pd.read_excel(prefix+'trabalhos2017v5.xlsx',index_col=0)

Insere duas colunas no lugar da Coluna de Autores

In [13]:
artsds["Afil"]=[geraAfil(record) for record in artsds["Autores"]]
artsds["Authors"]=[geraAuthors(record) for record in artsds["Autores"]]

In [14]:
artsds

,Modalidade,Área Temática,Título,Resumo,Funding,Autores,Apresentador,Formação Acadêmica,Instituição,Nacionalidade,Situação,Justificativas,Apresentado,Submeteu Trabalho Completo,Poster Code,Subm ID,Afil,Authors
Número,,,,,,,,,,,,,,,,,,
53695,Poster,RNA and Transcriptomics,High throughput sequencing of small RNAs in Bi...,Biomphalaria glabrata is a mollusc intermediat...,"UFU, FAPEMIG, CNPq and CAPES","Laysa Gomes Portilho (UFU),Fábio Ribeiro Queir...",Laysa Gomes Portilho,Graduação,UFU,Brasil,Aprovado,<justificativa>The abstract is fine. // </just...,Não,Não,TR14,53695,"[UFU, UFU, UFU, UFU, UFU, UFU, UFU]","[Laysa Gomes Portilho, Fábio Ribeiro Queiroz, ..."
53697,Poster,Genes and Genomics,Bioinformatic Analysis of Ubiquitin-Specific P...,The Bean is a leguminous plant with high prote...,"FAPEMIG, CNPq, UFU and CAPES","Monize Angela de Andrade (UFMG), Daniel Alexan...",Monize Angela de Andrade,Mestrado,UFU,Brasil,Aprovado,<justificativa>The abstract is fine. // </just...,Não,Não,GE22,53697,"[UFMG, UFMG, UFMG, UFMG, UFMG, UFMG]","[Monize Angela de Andrade, Daniel Alexandre Az..."
53699,Poster,Genes and Genomics,Acylsugar pathway in Solanum lycopersicum and ...,"The cultivated tomato, Solanum lycopersicum, i...","FAPEMIG, CNPq, UFU and CAPES","Thaís Cunha de Sousa Cardoso (UFU),Carolina Mi...",Thaís Cunha de Sousa Cardoso,Doutorado,Universidade Federal de Uberlandia - UFU,Brasil,Aprovado,<justificativa>The abstract is fine. // </just...,Não,Não,GE32,53699,"[UFU, UFU, UFU, UFU, UFU, UFU, UFU, UFU, UFU, ...","[Thaís Cunha de Sousa Cardoso, Carolina Milagr..."
53706,Poster,Phylogeny and Evolution,Genome-wide identification of novel miRNAs in ...,Cnidarian is a phylum of the kingdom Animalia ...,"FAPEMIG, CNPq, UFU and CAPES","Tamires Caixeta Alves (UFU),Laurence Rodrigues...",Tamires Caixeta Alves,Mestrado,UFU,Brasil,Aprovado,<justificativa></justificat,Não,Não,PE12,53706,"[UFU, UFU, UFU]","[Tamires Caixeta Alves, Laurence Rodrigues do ..."
53712,Poster,RNA and Transcriptomics,PRELIMINARY ANALYSIS OF miRNAs IN THE GENOME O...,The production of citrus is a highlight in the...,"CNPq, FAPEMIG, INCTV","Douglas Santana (UFU),Tamires Caixeta Alves (U...",Douglas Santana,Graduação,UFU,Brasil,Aprovado,<justificativa>The work describes a typical bi...,Não,Não,TR05,53712,"[UFU, UFU, UFU, UFU]","[Douglas Santana, Tamires Caixeta Alves, Laure..."
53719,Poster,RNA and Transcriptomics,"IN SILICO IDENTIFICATION, CHARACTERIZATION AND...",Capsicum annuum var. Glabriusculum is a specie...,"FAPEMIG, CNPq, UFU and CAPES","Ailton Pereira Da Costa Filho (UFU),Monize Ang...",AILTON PEREIRA DA COSTA FILHO,Graduação,UFU,Brasil,Aprovado,<justificativa>The abstract is fine. // </just...,Não,Não,TR01,53719,"[UFU, UFU, UFU, UFU]","[Ailton Pereira da Costa Filho, Monize Angela ..."
53742,Poster,RNA and Transcriptomics,CHARACTERIZATION AND INDENTIFICATION OF MATURE...,The cultivated pepper Capsicum annuum L. (Zunl...,"FAPEMIG, UFU, CNPq and CAPES","Fernando Augusto Corrêa Queiroz Cançado (UFU),...",Fernando Augusto Corrêa Queiroz Cançado,Graduação,UFU,Brasil,Aprovado,<justificativa>This work describes a typical u...,Não,Não,TR07,53742,"[UFU, UFU, UFU, UFU]","[Fernando Augusto Corrêa Queiroz Cançado, Moni..."
53752,Poster,Proteins and Proteomics,Virtual Screening of potential inhibitors for ...,Natural antioxidants compounds have been assoc...,"FAPEMIG, CNPq, UFU and CAPES","Heitor Cappato (UFU),Nilson Nicolau Junior (UF...",Heitor Cappato,Graduação,UFU,Brasil,Aprovado,<justificativa>The abstract is fine. // </just...,Não,Não,PR10,53752,"[UFU, UFU, UFU]","[Heitor Cappato, Nilson Nicolau Junior, Foued ..."
53753,Poster,RNA and Transcriptomics,Identification and characterization of miRNAs ...,Cucumber (Cucumis sativus L.) is one of the ma...,"FAPEMIG, CNPq, UFU and CAPES","Júlia Silveira Queiroz (UFU),Núbia Carolina Pe...",Júlia Silveira Queiroz,Graduação,UFU,Brasil,Aprovado,<justificativa>The structure of the abstract i...,Não,Não,TR13,53753,"[UFU, UFU, UFU, UFU]","

In [15]:
artsds["Authors"]

Número
53695    [Laysa Gomes Portilho, Fábio Ribeiro Queiroz, ...
53697    [Monize Angela de Andrade, Daniel Alexandre Az...
53699    [Thaís Cunha de Sousa Cardoso, Carolina Milagr...
53706    [Tamires Caixeta Alves, Laurence Rodrigues do ...
53712    [Douglas Santana, Tamires Caixeta Alves, Laure...
53719    [Ailton Pereira da Costa Filho, Monize Angela ...
53742    [Fernando Augusto Corrêa Queiroz Cançado, Moni...
53752    [Heitor Cappato, Nilson Nicolau Junior, Foued ...
53753    [Júlia Silveira Queiroz, Núbia Carolina Pereir...
53789    [Carlos Bruno de Araujo, Juliana da Silva Vian...
54093    [Liliane Santana Oliveira Kashiwabara, Dolores...
55967    [Hugo Rody Vianna Silva, Luiz Orlando de Olive...
56205    [Takahiko Murayama, Tatsunori Nishimura, Kana ...
56287     [Denilson Oliveira Melo, Paulo Eduardo Ambrósio]
56312    [Anton Semenchenko, Guilherme Oliveira, A. P. ...
56359    [Clovis Ferreira dos Reis, Rodrigo Juliani Siq...
57291                            [Ravi Jose Trist

## Separação em Grupos

In [16]:
artsdsgrouped=artsds.groupby(["Modalidade"],axis=0)
artsdsposter=artsdsgrouped.get_group('Poster')
artsdsHT=artsdsgrouped.get_group('HighLight Tracks')


In [17]:
tes=artsds.loc[61409]
fixbin(tes["Resumo"],replacements)


'Plant-parasitic nematodes have a worldwide distribution. They are virtually able to infest any human-cultivated plant. Annual losses caused by nematodes on life-sustaining crops are estimated to exceed 14\\% of the production (approximately 65 billion \\euro of loss worldwide). Previously studies were responsible for major advances in the identification of genes and mechanisms responsible for plants response to the Meloidogyne, the root-knot nematode. Meloidogyne spp. are obligate endoparasites that maintain a biotrophic relationship with their hosts. During the infection root cells are differentiated into specialized giant feeding cells through the releasing of effector proteins. However, despite the continuing efforts to identify new effectors and plant resistance mechanisms, studies have shown that the repertoire of both systems is limited. Recently, researchers published strong evidence that small RNAs from a phytopathogenic fungus act as effectors. These small RNAs hijack the hos

## Gera os arquivos com os abstracts

In [ ]:
for index, record in artsdsposter.iterrows():
    author=geraStringAuthors(record)
    title=fixbin(record["Título"],replacements)
    afiliation=geraStringAfil(record["Afil"])
    print(index)
    if record["Modalidade"]=="Poster":
        abstract=fixbin(record["Resumo"],replacements)
    else:
        abstract=""
    funding=fixbin(record["Funding"],replacements)
    stringfileresumo=strlatex.format(title,author,afiliation,abstract,funding)
    f = open('papers/art'+str(record["Número"])+".tex", 'w')
    f.write(stringfileresumo)
    f.close()

## Gera os arquivos com os índices 

In [24]:
areas = sorted(list(set(artsdsposter["Área Temática"])))
groupareas=artsdsposter.groupby(["Área Temática"],axis=0)
compstr = "";
for area in areas:
    print(area)
    ds=groupareas.get_group(area)
    ds=ds.sort_values("Poster Code")
    sessionstr = "\\chapter{" + area + "}\n";
    for index, record in ds.iterrows():
        title=record["Título"]
        author=", ".join(record["Authors"])
        file="art"+str(index)
        indextemp=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Authors"]])
        indexstr=indextemp
        sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
    compstr = compstr + sessionstr
f = open('posters.tex', 'w')
f.write(compstr)
f.close()

Genes and Genomics
Phylogeny and Evolution
Proteins and Proteomics
RNA and Transcriptomics
Software Development and Databases
Systems Biology and Networks


In [26]:
compstr = "";
ds=artsdsHT
sessionstr=""
ds=ds.sort_values("Poster Code")
for index, record in ds.iterrows():
    title=record["Título"]
    author=", ".join(record["Authors"])
    file="art"+str(index)
    indextemp=",".join(["\\index{"+brnames(aut)+"}" for aut in record["Authors"]])
    indexstr=indextemp
    sessionstr=sessionstr+strlatex2.format(title,author,indexstr,file)
    compstr = compstr + sessionstr
f = open('papers.tex', 'w')
f.write(compstr)
f.close()